In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/4/47/Saguaro_Sunset.jpg"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(IMAGE_URL, headers=headers)

print("Status:", response.status_code)
print("Content-Type:", response.headers.get("Content-Type"))

image_bytes = response.content
img_pil = Image.open(BytesIO(image_bytes))

plt.imshow(img_pil)
plt.axis('off')
plt.show()


Status: 404
Content-Type: text/html; charset=utf-8
b'<!DOCTYPE html>\n<html lang="en">\n<meta charset="utf-8">\n<title>Wikimedia Error</title>\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; font: 15px/1.6 sans-serif; color: #333; }\n.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; display: flex; flex-direction: row; flex-wrap: wrap; }\n.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }\nimg { margin: 0 2em 2em 0; }\na img {'


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001D8ED7D5170>